# Data Management for Visualizations

0. Import Libraries, Ingest Data, Understand Information, Finalize Columns
1. Manage NaNs
2. Bin, Discretize, Group & Filter
3. Visualize

In [8]:
#Import Libraries
import pandas as pd
import plotly.express as px
import numpy as np

In [3]:
#Ingest Data
df = pd.read_csv('ForeignAssistanceData.csv')

In [4]:
#Understand Data
print("Data Type Info:")
print(df.info())
print("\n")
print("Summary Info:")
print(df.describe())
print("\n")
print("NaN Count Info:")
df.isnull().sum()

Data Type Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33329 entries, 0 to 33328
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   FiscalYearType  33329 non-null  object 
 1   FiscalYear      33329 non-null  int64  
 2   OUID            33329 non-null  int64  
 3   AccountName     33071 non-null  object 
 4   AgencyName      26776 non-null  object 
 5   OUName          26776 non-null  object 
 6   Category        25206 non-null  object 
 7   Sector          20614 non-null  object 
 8   Amount          20176 non-null  float64
dtypes: float64(1), int64(2), object(6)
memory usage: 2.3+ MB
None


Summary Info:
         FiscalYear          OUID        Amount
count  33329.000000  33329.000000  2.017600e+04
mean    2010.267185    258.916319  2.011940e+07
std        2.425583     94.942636  1.358357e+08
min     2006.000000    164.000000 -6.744100e+04
25%     2008.000000    188.000000  8.400000e+05
50%     20

FiscalYearType        0
FiscalYear            0
OUID                  0
AccountName         258
AgencyName         6553
OUName             6553
Category           8123
Sector            12715
Amount            13153
dtype: int64

In [5]:
#Understand the Information in the dataframe
for colvar in df.columns:
    if df[colvar].dtype=="object":
        print("Column: ", colvar)
        print(df[colvar].value_counts())
        print("\n")

Column:  FiscalYearType
2008 Request         2941
2010 Base            2540
2010 Request         2155
2007 Base            2152
2008 Base            2116
2009 Base            2104
2011 Base            2097
2011 Request         2062
2006 Base            1983
2009 Request         1931
2012 Base            1886
2012 Request         1854
2013 Request         1777
2013 Base            1658
2014 Request         1646
2015 Request         1623
2013 OCO Base         204
2009 Supplemental     155
2012 OCO Base         108
2008 Supplemental      97
2007 Supplemental      88
2015 OCO Request       66
2014 OCO Request       26
2013 OCO Request       21
2014 Base              16
2010 OCO Base          11
2012 OCO Request       11
2011 OCO Base           1
Name: FiscalYearType, dtype: int64


Column:  AccountName
Development Assistance                                       7573
Economic Support Fund                                        6178
Assistance for Europe                                     

In [9]:
#Pivot on Country/Category, Dimensions on Fiscal Year, Aggregate on Amount
pd.pivot_table(df,index=["OUName","Category"],values=["Amount"], columns=["FiscalYear"],aggfunc=[np.sum],fill_value=0)

sum                         \
                                              Amount                          
FiscalYear                                      2006       2007        2008   
OUName      Category                                                          
Afghanistan Democracy                              0          0           0   
            Economic Development           311846000  675691000  1527559000   
            Education and Social Services   51368000   76148000   202459000   
            Environment                     11200000   20000000     8039000   
            Health                          52347000  114672000   230203000   
...                                              ...        ...         ...   
Zimbabwe    Environment                            0          0           0   
            Health                          10973000   18001000    40493000   
            Humanitarian Assistance                0          0           0   
            Peace and Security                     0          0           0   
            Program Management                     0          0           0   

                                                                              \
                                                                               
FiscalYear                                      2009        2010        2011   
OUName      Category                                                           
Afghanistan Democracy                              0           0           0   
            Economic Development           978462000  2027942000  1908865000   
            Education and Social Services  367163000   452314000   355670000   
            Environment                     10000000    10040000    10000000   
            Health                         198614000   293342000   329863000   
...                                              ...         ...         ...   
Zimbabwe    Environment                            0           0           0   
            Health                          84850000    74204000   125619000   
            Humanitarian Assistance                0           0           0   
            Peace and Security                     0       40000       40000   
            Program Management                     0     4556000           0   

                                                                              \
                                                                               
FiscalYear                                       2012        2013       2014   
OUName      Category                                                           
Afghanistan Democracy                               0           0          0   
            Economic Development           1998139000  1429610000  501000000   
            Education and Social Services   352020000   265227000  110000000   
            Environment                       7252000           0          0   
            Health                          407760000   354294000  140000000   
...                                               ...         ...        ...   
Zimbabwe    Environment                             0      648000          0   
            Health                          180351000   199111000  110175000   
            Humanitarian Assistance                 0     2268000          0   
            Peace and Security                  25000           0          0   
            Program Management                      0           0          0   

                                                      
                                                      
FiscalYear                                      2015  
OUName      Category                                  
Afghanistan Democracy                              0  
            Economic Development           299000000  
            Education and Social Services  137000000  
            Environment                            0  
            Health   

In [ ]:
#0. Drop less necessary columns
df.drop(['FiscalYearType'], axis=1, inplace=True)
df.drop(['AgencyName'], axis=1, inplace=True)

In [ ]:
#1. Manage NaNs
dfclean = df[df.AccountName.notnull() & 
    df.OUName.notnull() & 
      df.Category.notnull() &
      df.Sector.notnull()]

dfclean.Amount.fillna(0, inplace=True)
dfclean.isnull().sum()

In [ ]:
#3.  Binning
#Create Bin Labels for Amounts (lowest to highest in funding)
AmtBinLabel = ['Stone', 'Iron', 'Bronze', 'Silver', 'Gold', 'Platinum', 'Unbotanium']
AmtBins = [0, 500000, 1000000, 2000000, 5000000, 10000000, 25000000, 50000000]

#Evaluate Amount Column and print Output in readable format
pd.options.display.float_format = '${:,.2f}'.format
print(dfclean.Amount.describe())
print('\nWithout custom bins:')
print(pd.cut(dfclean['Amount'], bins=7).value_counts())
print(pd.cut(dfclean['Amount'], bins=7, labels=AmtBinLabel).value_counts())
print('\nWITH custom bins:')
print(pd.cut(dfclean['Amount'], bins=AmtBins).value_counts())
print(pd.cut(dfclean['Amount'], bins=AmtBins, labels=AmtBinLabel).value_counts())

In [ ]:
#3.  Group and Filter
#Filter for 2006 and then Group by Dimensions & Sum Amount
dfclean[dfclean['FiscalYear']==2006].groupby(['OUName','FiscalYear','Category']).sum()

#Use variables in lists to filter the data.
ConstCountries = ['Afghanistan', 'Brazil', 'Mexico','India']
ConstFY = [2006, 2007, 2008, 2009, 2010]

dfclean.OUName.isin(ConstCountries) #Array of booleans that match
df_orig = dfclean[dfclean.OUName.isin(ConstCountries) & dfclean.FiscalYear.isin(ConstFY)] #Find the specific Countries and Years
dfclean[~(dfclean.OUName.isin(ConstCountries) & dfclean.FiscalYear.isin(ConstFY))] #Reverse the mask.  Find everything ELSE.
plot_data2 = df_orig.groupby(['OUName','Category'])['Amount'].sum().groupby(level=[1]).cumsum() #Running total for each row(LVL=1)

#Putting it all together
dfclean[dfclean.OUName.isin(ConstCountries) & dfclean.FiscalYear.isin(ConstFY)].groupby(['OUName','FiscalYear','Category'])['Amount'].sum()

In [ ]:
#4.  Visualize Scatter with Frames
fig = px.scatter(df_orig, 
                  x=df_orig.Amount, 
                  y=df_orig.Sector, 
                  color=df_orig.Amount, 
                  hover_name=df_orig.OUName,
                  facet_col=df_orig.OUName,
                  animation_frame='FiscalYear')
fig.show()

In [ ]:
#4.  Visualize Scatter with Frames with Log Scale Data
fig = px.scatter(df_orig, 
                  x=df_orig.Amount, 
                  y=df_orig.Sector, 
                  color=df_orig.Amount, 
                  hover_name=df_orig.OUName,
                  facet_col=df_orig.OUName,
                  animation_frame='FiscalYear',
                  log_x=True) #RESCALE THE DATA
fig.show()

In [ ]:
#4.  Visualize 3D
fig = px.scatter_3d(df_orig, 
                     x='OUName', 
                     y='Sector', 
                     z='Amount',
                     color='Amount')
fig.show()